In [1]:
%pip install transformers huggingface_hub python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("hf_token2")

login(token=hf_token)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
import json

# Load the JSON data
with open('../dataset-generation/data/mintaka_test_extended.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

lang = 'da'
# [{id: "blabla", question: "yaya", label: "xdd"}]

local_question_answer = []
for question in data:
    qa_entity = {
            "id": question['id'],
            "question": question['translations'][lang],
        }
    answer = question['answer']
    if answer['answerType'] in ["numerical", "boolean", "date", "string"]:
        qa_entity['answer'] = answer['answer'][0]
    elif answer['answer'] == None:
        continue
    elif answer['answer'][0]['label'][lang] != None:
        qa_entity['answer'] = answer['answer'][0]['label'][lang]
    else:
        continue
    local_question_answer.append(qa_entity)


print(local_question_answer[0])

{'id': 'fae46b21', 'question': 'Hvilken mand var en berømt amerikansk forfatter og også dampskibspilot på Mississippi-floden?', 'answer': 'Mark Twain'}


In [4]:
from transformers import pipeline

# pretrained_model = "meta-llama/Llama-3.3-70B-Instruct"
pretrained_model = "meta-llama/Llama-3.2-1B-Instruct"

generator = pipeline('text-generation', model=pretrained_model)



OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Llama-3.2-1B-Instruct is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
results = []

# prompt = "Svar kort: \n" + local_question_answer[2]['question']
# generated_text = generator(prompt, max_length=50, num_return_sequences=5)
# print(generated_text)

# qa_result = local_question_answer[2]
# qa_result['model_answer'] = generated_text
# results.append(qa_result)

# [{'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \nSvar: \nThe Weeknd (2015) \nDrake (2016) \n\nHvis du vil vide mere om denne spørg'},
#  {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nSvar: \nHvis man ser på følgende liste, så er The Weeknd ældst af de to. Han følger'},
#   {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \nHvis du spørger om The Weeknd, er han ældst af de to. \nHvis du spørger om Drake,'},
#   {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \nHvis du spørger om ældst, er Drake ældst, men The Weeknd er også ældst.\n\nJeg er'},
#   {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nSvar: \nHvis man ligger i orden, så er The Weeknd ældst. Han født i 1990, mens'}]

for entry in local_question_answer:
    prompt = "Svar kort: \n" + question['question']
    generated_text = generator(prompt, max_length=35, num_return_sequences=5)
    qa_result = entry
    qa_result['model_answer'] = generated_text

    results.append(qa_result)

import json

output_file = "results_" + pre_trained_model + '_' + lang + '.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nSvar: \nHvis du spørger om ældste af dem, er det The Weeknd som er ældst. Han fø'}, {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \nHvis du spørger om The Weeknd, er han ældst af de to, født i 1990. \nHvis'}, {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nSvar: \nDe to er alle ældste af medlemmerne i gruppen The Weeknd (født 24. m'}, {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nSvar: \nHvis du spørger om ældst af disse tre, så er The Weeknd 32 år, Drake 31'}, {'generated_text': 'Svar kort: \nHvem er ældst, The Weeknd eller Drake? \n\nEfter at have undersøbt, er The Weeknd 33 år gammel og Drake 30 år gammel. \n\nSvar: The'}]
